In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import boto3
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.tuner import HyperparameterTuner, IntegerParameter, ContinuousParameter

## Criando Base de Dados

In [ ]:
# definindo os parâmetros para a base de dados
np.random.seed(0)
n_samples = 10000
area = np.random.normal(150, 50, n_samples)
num_rooms = np.random.randint(1, 6, n_samples)
num_bathrooms = np.random.randint(1, 4, n_samples)
age = np.random.randint(1, 50, n_samples)
distance_to_city_center = np.random.normal(10, 5, n_samples)
price = 100000 + (area * 1000) + (num_rooms * 5000) - (num_bathrooms * 3000) - (age * 1000) - (distance_to_city_center * 2000)

# criando o dataframe (importante deixar "price" - variavel predita - na primeira posicao)
data = pd.DataFrame({
    'price': price,
    'area': area,
    'num_rooms': num_rooms,
    'num_bathrooms': num_bathrooms,
    'age': age,
    'distance_to_city_center': distance_to_city_center
})

# salvando em csv 
data.to_csv('housing_data.csv', index=False)

### Definindo Bases de Treino, Teste e Validacao

In [ ]:
# configuração do bucket S3
bucket = 'hpo-data'
s3 = boto3.client('s3')

# normaliza os dados
scaler = StandardScaler()
normalized_data = scaler.fit_transform(data)

# converte de volta para DataFrame
normalized_data = pd.DataFrame(normalized_data, columns=data.columns)

# determina o número de linhas para cada conjunto (treino, teste e validacao)
train_size = int(0.7 * len(normalized_data))
test_size = int(0.2 * len(normalized_data))
validation_size = len(normalized_data) - train_size - test_size

# divide os dados nos tamanhos definidos acima
train_data = normalized_data[:train_size]
test_data = normalized_data[train_size:train_size + test_size]
validation_data = normalized_data[train_size + test_size:]

# salva os conjuntos de dados em arquivos CSV sem a header
train_data.to_csv('houses_train_data.csv', index=False, header=False)
test_data.to_csv('houses_test_data.csv', index=False, header=False)
validation_data.to_csv('houses_validation_data.csv', index=False, header=False)

# upload dos arquivos CSV para o bucket S3
s3.upload_file('houses_train_data.csv', bucket, 'houses_train_data.csv')
s3.upload_file('houses_test_data.csv', bucket, 'houses_test_data.csv')
s3.upload_file('houses_validation_data.csv', bucket, 'houses_validation_data.csv')

# caminhos S3 para os dados
train_data_path = f's3://{bucket}/houses_train_data.csv'
test_data_path = f's3://{bucket}/houses_test_data.csv'
validation_data_path = f's3://{bucket}/houses_validation_data.csv'

## Realizando Training Jobs (HPO)

In [ ]:
# configuracao da entrada de dados para treinamento, teste e validação
s3_input_train = TrainingInput(s3_data=train_data_path, content_type='text/csv')
s3_input_validation = TrainingInput(s3_data=validation_data_path, content_type='text/csv')
s3_input_test = TrainingInput(s3_data=test_data_path, content_type='text/csv')

# obtém o papel IAM
role = get_execution_role()

# criacao do estimador SageMaker com o script de treinamento do XGBoost
estimator = Estimator(entry_point='train_script.py',
                      role=role,
                      instance_count=1,
                      instance_type='ml.m5.large',
                      image_uri=sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "latest"),
                      sagemaker_session=sagemaker.Session())

# hiperparametros para otimizacao
hyperparameter_ranges = {
    'max_depth': IntegerParameter(3, 10),
    'eta': ContinuousParameter(0.01, 0.3),
    'min_child_weight': ContinuousParameter(1, 10),
    'subsample': ContinuousParameter(0.5, 0.9),
    'gamma': ContinuousParameter(0, 0.5),
    'num_round': IntegerParameter(50, 500)
}

# configuracao da otimizacao de hiperparametros
objective_metric_name = 'validation:rmse' 
objective_type = 'Minimize'
metric_definitions = [{'Name': 'validation:rmse', 'Regex': 'validation:rmse: ([0-9\\.]+)'}]

# define numero de jobs + quantos serao executados simultaneamente
max_jobs = 20
max_parallel_jobs = 2

# cria o tuner com a definicao da metrica
tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            objective_type=objective_type,
                            max_jobs=max_jobs,
                            max_parallel_jobs=max_parallel_jobs)
print(s3_input_test)

# inicia a hpo e fornece os dados de treinamento, teste e validacao
tuner.fit({'train': s3_input_train, 'test': s3_input_test, 'validation': s3_input_validation})

## Definindo Melhor Job

In [ ]:
import boto3
import pandas as pd
from rich import print

# conecta ao sagemaker
sagemaker_client = boto3.client('sagemaker')

# lista os training jobs (os ultimos 20)
response = sagemaker_client.list_training_jobs(
    MaxResults=20,  # número maximo de jobs 
    SortBy='CreationTime',  # ordena por data de criacao
    SortOrder='Descending'  # ordena em ordem decrescente para obter os jobs mais recentes primeiro
)

training_jobs = response['TrainingJobSummaries']

# extrai infos relevantes dos jobs
job_info = []
for job in training_jobs:
    job_name = job['TrainingJobName']
    job_arn = job['TrainingJobArn']
    job_status = job['TrainingJobStatus']
    
    # obtem metricas de desempenho se o job estiver concluido
    if job_status == 'Completed':
        metrics_response = sagemaker_client.describe_training_job(
            TrainingJobName=job_name
        )

        # extrai a metrica de interesse (nesse caso, rMSE)
        metric = metrics_response['FinalMetricDataList'][0]  # selecionando metricas (teste)
        metric_value = metric['Value']

        metric_V = metrics_response['FinalMetricDataList'][1]  # selecionando metricas (validacao)
        metric_value_V = metric_V['Value']
        
        job_info.append({'JobName': job_name, 'rmseTrain': metric_value, 'rmseValidation': metric_value_V})

# converte os resultados em um DataFrame pandas para facilitar a analise
df = pd.DataFrame(job_info)

# encontra melhor modelo com base na metrica
best_model_train = df[df['rmseTrain'] == df['rmseTrain'].min()]
best_model_validation = df[df['rmseValidation'] == df['rmseValidation'].min()]

print(f'Melhor modelo (treino): {best_model_train}')
print('')
print(f'Melhor modelo (validacao): {best_model_validation}')